In [1]:
library(data.table)

In [2]:
library(stringr)

In [3]:
# get variant information
# preprocessing
variant <- read.table("output/DNA__Exome_Seq_none.txt", skip = 13, sep = "\t", fill = TRUE)   
variant <- variant[,-77]
colnames <- "BR:MCF7	BR:MDA-MB-231	BR:HS 578T	BR:BT-549	BR:T-47D	CNS:SF-268	CNS:SF-295	CNS:SF-539	CNS:SNB-19	CNS:SNB-75	CNS:U251	CO:COLO 205	CO:HCC-2998	CO:HCT-116	CO:HCT-15	CO:HT29	CO:KM12	CO:SW-620	LE:CCRF-CEM	LE:HL-60(TB)	LE:K-562	LE:MOLT-4	LE:RPMI-8226	LE:SR	ME:LOX IMVI	ME:MALME-3M	ME:M14	ME:SK-MEL-2	ME:SK-MEL-28	ME:SK-MEL-5	ME:UACC-257	ME:UACC-62	ME:MDA-MB-435	ME:MDA-N	LC:A549/ATCC	LC:EKVX	LC:HOP-62	LC:HOP-92	LC:NCI-H226	LC:NCI-H23	LC:NCI-H322M	LC:NCI-H460	LC:NCI-H522	OV:IGROV1	OV:OVCAR-3	OV:OVCAR-4	OV:OVCAR-5	OV:OVCAR-8	OV:SK-OV-3	OV:NCI/ADR-RES	PR:PC-3	PR:DU-145	RE:786-0	RE:A498	RE:ACHN	RE:CAKI-1	RE:RXF 393	RE:SN12C	RE:TK-10	RE:UO-31"
colname <- strsplit(colnames, split = "\t", fixed = TRUE)
colnames(variant)[17:76] <- colname[[1]]
colnames(variant)[1:16] <-  variant[1,][1:16]
variant <- variant[-1,]
rownames(variant) <- 1:nrow(variant)
write.table(variant, "variant.csv", row.names = FALSE)


In [4]:
# use cutoff on the variant allele frequency to define presence of a variant in a given sample,  > 0 

In [5]:
variant <- fread("variant.csv")
# extract rare variants of the following classes: missense, stop, frameshift, splice
select_variant <- variant[variant$`SNP Type (l)` %in% c('Missense', 'Frameshift', 'Nonsense', 'Splicing'),]
variant_sample <- select_variant[,17:ncol(variant)]
variant_sample$Identifier <- select_variant$`Identifier (c)`
nrow(variant_sample)
head(variant_sample)

[1] 57458

BR:MCF7,BR:MDA-MB-231,BR:HS 578T,BR:BT-549,BR:T-47D,CNS:SF-268,CNS:SF-295,CNS:SF-539,CNS:SNB-19,CNS:SNB-75,⋯,PR:DU-145,RE:786-0,RE:A498,RE:ACHN,RE:CAKI-1,RE:RXF 393,RE:SN12C,RE:TK-10,RE:UO-31,Identifier
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0,0,0,0,0,0,0,0.000,0,0,⋯,0,0,0,0,0,0,85.714,0,0,chr1:865628_G_A
0,0,0,0,0,0,0,0.000,0,0,⋯,0,0,0,0,0,0,0.000,0,0,chr1:871199_G_A
0,0,0,0,0,0,0,0.000,0,0,⋯,0,0,0,0,0,0,0.000,0,0,chr1:874788_C_G
0,0,0,0,0,0,0,0.000,0,0,⋯,0,0,0,0,0,0,0.000,0,0,chr1:874816_*_+T
0,0,0,0,0,0,0,0.000,0,0,⋯,0,100,0,0,0,0,0.000,100,0,chr1:877831_T_C
0,0,0,0,0,0,0,66.667,0,0,⋯,0,0,0,0,0,0,0.000,0,0,chr1:878331_C_T


In [6]:
#use cutoff on the variant allele frequency to define presence of a variant in a given sample (start with > 0)
variant_sample_cutoff <- melt(variant_sample, variable.name = "sample(cell_line)", value.name = "frequency")
variant_sample_cutoff <- variant_sample_cutoff[variant_sample_cutoff$frequency > 0,]
head(variant_sample_cutoff)
nrow(variant_sample_cutoff)

Warning message in melt.data.table(variant_sample, variable.name = "sample(cell_line)", :
“id.vars and measure.vars are internally guessed when both are 'NULL'. All non-numeric/integer/logical type columns are considered id.vars, which in this case are columns [Identifier, ...]. Consider providing at least one of 'id' or 'measure' vars in future.”


Identifier,sample(cell_line),frequency
<chr>,<fct>,<dbl>
chr1:888659_T_C,BR:MCF7,100.000
chr1:1120431_G_A,BR:MCF7,98.182
chr1:1269554_T_C,BR:MCF7,91.667
chr1:1558792_T_C,BR:MCF7,100.000
chr1:1686040_G_T,BR:MCF7,100.000
chr1:1847979_T_C,BR:MCF7,100.000


[1] 352827

In [7]:
variant_list <- as.data.frame(unique(variant_sample_cutoff$Identifier))
colnames(variant_list) <- "Identifier"
variant_list[,c("chr", "pos")] <- as.data.frame(str_split_fixed(variant_list$Identifier, ':', 2))
variant_list[,c("begin", "ref", "alt")] <- as.data.frame(str_split_fixed(variant_list$pos, '_', 3))
variant_list$chr <- gsub('chr','', variant_list$chr)
variant_list$snp <- paste0(variant_list$ref, "/", variant_list$alt)
variant_list$end <- variant_list$begin
write.table(variant_list[, c("chr", "begin", "end", "snp")], "variant_list.tsv", sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE)
head(variant_list[, c("chr", "begin", "end", "snp")])
nrow(variant_list)

,chr,begin,end,snp
,<chr>,<chr>,<chr>,<chr>
1,1,888659,888659,T/C
2,1,1120431,1120431,G/A
3,1,1269554,1269554,T/C
4,1,1558792,1558792,T/C
5,1,1686040,1686040,G/T
6,1,1847979,1847979,T/C


[1] 57458

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
options(timeout = 3000) 
BiocManager::install("MafDb.gnomAD.r2.1.hs37d5")

In [8]:
library("MafDb.gnomAD.r2.1.hs37d5")
mafh5 <- MafDb.gnomAD.r2.1.hs37d5

Lade nötiges Paket: GenomicScores

Lade nötiges Paket: S4Vectors

Lade nötiges Paket: stats4

Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min



Attache Paket: ‘S4Vectors’


Die folgenden Objekte sind maskiert von ‘package:data.table’:

    first, second


Die folgenden Objekte sind maskiert von ‘package:base’:

    I, expand.grid, unname


Lade nötiges Paket: GenomicRanges

Lade nötiges Paket: IRanges


Attache Paket: ‘IRanges’


D

In [9]:
gr <- GRanges(seqnames = variant_list$chr, 
              ranges = IRanges(start = as.numeric(variant_list$begin), width = 1))
annotate <- gscores(mafh5, gr)
annotate_res <- as.data.frame(annotate)
head(annotate_res)
nrow(annotate_res)

no MafDb scores for population AF in sequence Y from GScores object x (MafDb.gnomAD.r2.1.hs37d5).



,seqnames,start,end,width,strand,AF
,<fct>,<int>,<int>,<int>,<fct>,<dbl>
1,1,888659,888659,1,*,0.07
2,1,1120431,1120431,1,*,0.13
3,1,1269554,1269554,1,*,0.03
4,1,1558792,1558792,1,*,0.14
5,1,1686040,1686040,1,*,0.31
6,1,1847979,1847979,1,*,0.14


[1] 57458

In [10]:
annotate_res <- na.omit(annotate_res )
nrow(annotate_res)
annotate_res <- annotate_res[!duplicated(annotate_res$start),]
nrow(annotate_res)

[1] 31261

[1] 31198

In [11]:
annotate_res <- merge(variant_list, annotate_res, all.x = TRUE)
annotate_res <- annotate_res[,c("Identifier", "AF")]
annotate_res[is.na(annotate_res)] <- 0.00000001
annotate_res <- annotate_res[annotate_res$AF < 0.01,]
head(annotate_res)
nrow(annotate_res)

,Identifier,AF
,<chr>,<dbl>
1,chr7:100001379_C_A,1e-08
3,chr7:100004867_T_C,3e-03
4,chr2:100006292_A_G,1e-08
5,chr2:100007061_C_T,6e-05
6,chr7:100007167_C_A,4e-04
7,chr10:100010849_G_A,1e-08


[1] 39625

In [12]:
variant_with_maf <- merge(variant_sample_cutoff , annotate_res)
nrow(variant_with_maf)
head(variant_with_maf)

[1] 68571

Identifier,sample(cell_line),frequency,AF
<chr>,<fct>,<dbl>,<dbl>
chr10:100010849_G_A,ME:SK-MEL-2,45.238,1e-08
chr10:100013418_T_G,ME:SK-MEL-2,56.140,6e-05
chr10:100017532_G_T,LC:EKVX,82.456,9e-03
chr10:100020867_C_A,CO:HCT-15,38.889,1e-08
chr10:100021911_C_T,BR:MCF7,29.508,2e-04
chr10:100167409_A_C,CO:HCC-2998,44.000,1e-08


In [13]:
write.csv(variant_with_maf, quote=FALSE, "variant_with_maf.csv")